In [46]:
import pandas as pd 

def dateToInt(date):
	res = date.split('/')
	return int(res[0])*30 + int(res[1]) + int(res[2])*365

data = pd.read_csv('data.csv')
data = data.dropna()
data['dateInDays'] = data['Date'].apply(lambda x: dateToInt(x))
data['lbd_in_days'] = data['Last Breeding Date'].apply(lambda x: dateToInt(x))
data['lbd'] = data['dateInDays'] - data['lbd_in_days']
data = data.sort_values(by=['Cow Number', 'dateInDays'])

data.head()

,Unnamed: 0,Cow Number,Date,Lactation Number,Lactation Status,Days in Lactation,Daily Production,Last Breeding Date,dateInDays,lbd_in_days,lbd
43589,43590,7354,11/30/2018,2,Pregnant,316.0,56.9,6/8/2018,736930,736758,172
43590,43591,7354,12/1/2018,2,Pregnant,317.0,56.4,6/8/2018,736931,736758,173
43591,43592,7354,12/2/2018,2,Pregnant,318.0,59.0,6/8/2018,736932,736758,174
43592,43593,7354,12/3/2018,2,Pregnant,319.0,58.7,6/8/2018,736933,736758,175
43593,43594,7354,12/4/2018,2,Pregnant,320.0,57.4,6/8/2018,736934,736758,176


In [47]:
data = data.sort_values(by=['Cow Number', 'dateInDays'])
data['day_i'] = data.groupby(['Cow Number']).cumcount()+1
data.head()

,Unnamed: 0,Cow Number,Date,Lactation Number,Lactation Status,Days in Lactation,Daily Production,Last Breeding Date,dateInDays,lbd_in_days,lbd,day_i
43589,43590,7354,11/30/2018,2,Pregnant,316.0,56.9,6/8/2018,736930,736758,172,1
43590,43591,7354,12/1/2018,2,Pregnant,317.0,56.4,6/8/2018,736931,736758,173,2
43591,43592,7354,12/2/2018,2,Pregnant,318.0,59.0,6/8/2018,736932,736758,174,3
43592,43593,7354,12/3/2018,2,Pregnant,319.0,58.7,6/8/2018,736933,736758,175,4
43593,43594,7354,12/4/2018,2,Pregnant,320.0,57.4,6/8/2018,736934,736758,176,5


In [48]:
data.columns

Index(['Unnamed: 0', 'Cow Number', 'Date', 'Lactation Number',
       'Lactation Status', 'Days in Lactation', 'Daily Production',
       'Last Breeding Date', 'dateInDays', 'lbd_in_days', 'lbd', 'day_i'],
      dtype='object')

In [49]:
def getLabels(lactationStatus):
    return lactationStatus=='Pregnant'

data['target'] = data['Lactation Status'].apply(lambda x: getLabels(x))

In [50]:
data['target'].value_counts()

True     39078
False    31766
Name: target, dtype: int64

In [53]:
data = data[['Cow Number', 'Lactation Number', 'Daily Production', 'day_i','lbd', 'target']]

In [54]:
data.head()

,Cow Number,Lactation Number,Daily Production,day_i,lbd,target
43589,7354,2,56.9,1,172,True
43590,7354,2,56.4,2,173,True
43591,7354,2,59.0,3,174,True
43592,7354,2,58.7,4,175,True
43593,7354,2,57.4,5,176,True
